In [27]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import glob

In [28]:
CSV_DIR = "dataframes"

In [38]:
def evaluate_model_metrics_text(folder_path):
    """
    Process all CSV files in the specified folder, computing confusion matrices
    and classification metrics (precision, recall, F1-score, support) for each file.

    Parameters:
        folder_path (str): Path to the folder containing CSV files.

    Returns:
        None
    """
    # Get all CSV files in the folder and sort them numerically
    csv_files = sorted(
        glob.glob(os.path.join(folder_path, "*.csv")),
        key=lambda x: int(os.path.basename(x).split("(")[1].split(")")[0]) if "(" in os.path.basename(x) else float("inf")
    )

    if not csv_files:
        print("No CSV files found in the folder.")
        return

    for file in csv_files:
        file_name = os.path.basename(file)
        
        print(f"\nProcessing file: {file_name}")

        # Skip file if the name matches trial (X).csv format
        if not (file_name.startswith("trial") and "(" in file_name and ")" in file_name):
            continue

        # Read the CSV file
        df = pd.read_csv(file)

        # Ensure the required columns exist
        if not {'Damage Type', 'Prediction'}.issubset(df.columns):
            continue

        # Extract ground truth and predictions
        y_true = df['Damage Type']
        y_pred = df['Prediction']

        # Compute confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        print("Confusion Matrix:")
        print(cm)

        # Compute classification metrics
        report = classification_report(y_true, y_pred, zero_division=0)
        print("Classification Report:")
        print(report)


In [39]:
evaluate_model_metrics_text(CSV_DIR)


Processing file: trial (2).csv
Confusion Matrix:
[[147  27  19]
 [  8 231  28]
 [ 17  42 177]]
Classification Report:
                precision    recall  f1-score   support

Corrosionstain       0.85      0.76      0.81       193
        Cracks       0.77      0.87      0.81       267
      Spalling       0.79      0.75      0.77       236

      accuracy                           0.80       696
     macro avg       0.80      0.79      0.80       696
  weighted avg       0.80      0.80      0.80       696


Processing file: trial (3).csv
Confusion Matrix:
[[157   9  27]
 [  3 236  28]
 [ 20  23 193]]
Classification Report:
                precision    recall  f1-score   support

Corrosionstain       0.87      0.81      0.84       193
        Cracks       0.88      0.88      0.88       267
      Spalling       0.78      0.82      0.80       236

      accuracy                           0.84       696
     macro avg       0.84      0.84      0.84       696
  weighted avg       0.84    